In [206]:
import pandas as pd

In [207]:
rationales = pd.read_excel('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] RATIONALES/Contrastive (1).xlsx', index_col=0)

In [208]:
rationales.rationales[15].lower().split('most likely answer: ')

['6. the most likely answer is (e) gather flowers because if a child is in a meadow in spring, they might engage in activities like picking or gathering flowers. the other choices (clean room, set table, play with toys, form opinions) are either unrelated or not specific to being in a meadow.']

In [209]:
rationales['split'] = rationales.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [210]:
rationales.split = rationales.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [211]:
import ast

# convert choices to list
rationales['choices'] = rationales.choices.apply(lambda x: ast.literal_eval(x))

In [212]:
# get the index of the correct answer
import numpy as np
rationales['correct_index'] = [{} for _ in range(len(rationales))]
for index, row in rationales.iterrows():
    for choice in row['choices']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].index(choice)

In [213]:
# get the key of value min
rationales['LLM_answer'] =rationales.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [214]:
# HANDLE EXCEPTION
error = rationales[rationales.LLM_answer.isna()]
error.head(5)

,question,choices,answer,rationales,split,correct_index,LLM_answer
182,After the guilty verdict in the killing the ju...,"[wrong, death of, bad, negative, evil]",evil,"3. After the guilty verdict in the killing, th...",because it reflects the severe judgment and c...,{},NaN
183,After the guy is successful in cashing in his ...,"[sad, quitting, extra money, great joy, leave]",great joy,4. After the guy is successful in cashing in h...,because it is a common emotional response to ...,{},NaN
185,"After the lunch break, what did the men resume...","[working, fix it, accelerate, putting together...",working,"6. After the lunch break, the men resumed (A) ...",because it is a common activity one would ret...,{},NaN
278,"Bill was awaking, what was happening to him ph...","[tiredness, grogginess, discomfort, getting ou...",eyes opening,9. (E) eyes opening: The question asks what is...,is that his eyes are opening.,{},NaN
340,"By laughing while playing, what were the child...","[learning, getting tired, injuries, having fun...",having fun,1. (D) having fun: The fact that the children ...,because it directly aligns with the behavior ...,{},NaN


In [215]:
# get the index of the correct answer
import numpy as np
error['correct_index'] = [{} for _ in range(len(error))]
for index, row in error.iterrows():
    for choice in row['choices']:
        if choice in row['rationales']:
            row['correct_index'][choice] = row['rationales'].index(choice)

<ipython-input-215-25441d68bc9a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['correct_index'] = [{} for _ in range(len(error))]


In [216]:
# get the key of value min
error['LLM_answer'] = error.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

<ipython-input-216-1f6358438110>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['LLM_answer'] = error.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)


In [217]:
print(error[error.LLM_answer.isna()].question)

Series([], Name: question, dtype: object)


In [220]:
rationales.LLM_answer = rationales.LLM_answer.fillna(error.LLM_answer)

In [224]:
(sum(rationales.LLM_answer == rationales.answer)) / len(rationales)

0.7416529830322933